In [1]:
import horton
import numpy as np
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [17]:
def get_free_atom_energy(basisset, Z, spins):
    horton.log.set_level(0)
    mol = horton.IOData()
    mol.coordinates = np.array([[0.0, 0.0, 0.0],])
    mol.numbers = np.array([Z,])
    mol.pseudo_numbers = np.array([Z,]) 
    
    # build basis set
    obasis = horton.get_gobasis(mol.coordinates, np.array([int(Z),]), basisset)
    if int(Z) != 7:
        obasis2 = horton.get_gobasis(mol.coordinates, np.array([7,]), basisset)
        obasis = horton.GOBasis.concatenate(obasis, obasis2)
    olp = obasis.compute_overlap()
    kin = obasis.compute_kinetic()
    na = obasis.compute_nuclear_attraction(mol.coordinates, mol.pseudo_numbers)
    er = obasis.compute_electron_repulsion()
    
    orb_alpha = horton.Orbitals(obasis.nbasis)
    orb_beta = horton.Orbitals(obasis.nbasis)
    
    one = kin + na
    horton.guess_core_hamiltonian(olp, one, orb_alpha, orb_beta)
    
    external = {'nn': horton.compute_nucnuc(mol.coordinates, mol.pseudo_numbers)}
    
    terms = [horton.UTwoIndexTerm(kin, 'kin'), horton.UDirectTerm(er, 'hartree'), horton.UExchangeTerm(er, 'x_hf'), horton.UTwoIndexTerm(na, 'ne')]
    ham = horton.UEffHam(terms, external)
    occ_model = horton.AufbauOccModel(spins[0], spins[1])
    occ_model.assign(orb_alpha, orb_beta)
    dm_alpha = orb_alpha.to_dm()
    dm_beta = orb_beta.to_dm()
    scf_solver = horton.EDIIS2SCFSolver(1e-5, maxiter=400)
    scf_solver(ham, olp, occ_model, dm_alpha, dm_beta)
    
    fock_alpha = np.zeros(olp.shape)
    fock_beta = np.zeros(olp.shape)
    ham.reset(dm_alpha, dm_beta)
    energy = ham.compute_energy()    
    return {'basis': basisset, 'Z': Z, 'energy': energy}
spins = {1:(1,0),2:(1,1),3:(2,1),4:(2,2),5:(2,3), 6:(4, 2), 7:(5, 2), 8: (5, 3), 9:(5,4), 10:(5,5), 11:(6,5), 12:(6,6), 13:(7,6)}
atom_energies = [get_free_atom_energy('STO-3G', _, spins[_]) for _ in spins.keys()]
atom_energies += [get_free_atom_energy('def2-tzvp', _, spins[_]) for _ in spins.keys()]

In [19]:
df = pd.DataFrame(atom_energies); df

,Z,basis,energy
0,1,STO-3G,-0.479167
1,2,STO-3G,-2.849022
2,3,STO-3G,-7.284902
3,4,STO-3G,-14.082978
4,5,STO-3G,-24.247894
5,6,STO-3G,-37.243679
6,7,STO-3G,-53.719010
7,8,STO-3G,-73.825481
8,9,STO-3G,-98.029318
9,10,STO-3G,-126.703121


In [20]:
df.to_pickle('atom-energies.pkl')